In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from boxoffice.db.frames import get_movie_frame_full
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [101]:
target_slugs = ['Paddington-2', 'Deadpool-and-Wolverine-(2024)', 'Renfield']

df = get_movie_frame_full()

Reading from movies.csv


In [102]:
# make dummy variables for genre, mpaa_rating, source, production_method, creative_type
# eventually include franchise
columns = []
for col in ['genre', 'mpaa_rating', 'source', 'production_method', 'creative_type']:
    dummies = pd.get_dummies(df[col], prefix=col)
    columns.extend(dummies.columns)
    df = pd.concat([df, dummies], axis=1)
columns.extend(['budget'])

In [103]:
# make na budget 0
df_distances = df[columns]

print('Budget NA:', df['budget'].isna().sum())

# print all the NA titles in budget
na_title_df = df[df['budget'].isna()]
with open('na_budget_titles.txt', 'w') as f:
    for row in na_title_df.iterrows():
        f.write(row[1]['title'] + ' ' + row[1]['slug'] + '\n')

df_distances['budget'] = df_distances['budget'].fillna(-1)

# divide budget by 1 million
df_distances['budget'] = df_distances['budget'] / 20000000

# multiply all the mpaas by 10
# df_distances['mpaa_rating'] = df_distances['mpaa_rating'].apply(lambda x: x * 10)

euclidean_dist = euclidean_distances(df_distances)

def format_number_in_millions(x):
    return "{:.2f}M".format(x / 1000000)

for target_slug in target_slugs:
    # find the row where the id is the target
    target = df[df['slug'] == target_slug]
    target_id = target.index[0]
    target = target[columns]
    target = target.values.reshape(1, -1)
    distances = euclidean_dist[target_id]
    closest = np.argsort(distances)
    print('Target:', target_id, df.loc[target_id]['title'])
    for i in range(0, 11):
        located = df.loc[closest[i]]
        print(i, located['title'], round(distances[closest[i]], 2), format_number_in_millions(located['total_revenue_within_365_days']))
    print()

Budget NA: 230
Target: 716 Paddington 2
0 Paddington 2 0.0 40.77M
1 Harold and the Purple Crayon 0.0 18.32M
2 Peter Rabbit 2: The Runaway 0.25 41.24M
3 Peter Rabbit 0.5 115.23M
4 Paddington 0.75 76.40M
5 Clifford the Big Red Dog 1.2 48.95M
6 The Nut Job 2: Nutty by Nature 1.41 28.70M
7 Goosebumps 2: Haunted Halloween 1.44 47.42M
8 The Ant Bully 1.44 20.84M
9 Nanny McPhee and the Big Bang 1.44 29.01M
10 The Hitchhiker's Guide to the Galaxy 1.44 48.14M

Target: 0 Deadpool & Wolverine
0 Deadpool & Wolverine 0.0 662.89M
1 Iron Man 3 1.41 405.84M
2 Spider-Man: No Way Home 1.41 864.80M
3 Guardians of the Galaxy Vol 2 1.41 406.81M
4 Green Lantern 1.41 116.60M
5 Ant-Man and the Wasp: Quantumania 1.41 232.01M
6 Doctor Strange in the Multiverse of Madness 1.41 447.33M
7 Suicide Squad 1.89 345.60M
8 The Avengers 1.89 637.75M
9 Captain Marvel 1.89 447.53M
10 The Amazing Spider-Man 2 2.0 211.29M

Target: 228 Renfield
0 Renfield 0.0 18.05M
1 Safe House 1.42 126.15M
2 It’s Complicated 1.42 112.74M
3 

/tmp/ipykernel_26340/2335400341.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_distances['budget'] = df_distances['budget'].fillna(-1)
/tmp/ipykernel_26340/2335400341.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_distances['budget'] = df_distances['budget'] / 20000000
